In [1]:
! whoami

root


In [2]:
! ls

bin	   dev	   home   lib64       media  proc  sbin  sys  var
boot	   etc	   lib	  libx32      mnt    root  snap  tmp
copyright  hadoop  lib32  lost+found  opt    run   srv	 usr


In [6]:
! whoami
! hdfs dfs -ls /
! hdfs dfs -ls /user
! hdfs dfs -mkdir /user/adta5240grp

root
Found 3 items
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:28 /tmp
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:30 /user
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /var
Found 13 items
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /user/dataproc
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /user/hbase
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /user/hdfs
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /user/hive
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /user/kafka
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /user/mapred
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /user/pig
drwxr-xr-x   - root hadoop          0 2024-04-16 23:30 /user/root
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /user/solr
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /user/spark
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23:27 /user/yarn
drwxrwxrwt   - hdfs hadoop          0 2024-04-16 23

In [7]:
! hdfs dfs -mkdir /user/adta5240grp/data

In [8]:
! mkdir data

In [9]:
! cd data

In [16]:
! gsutil cp gs://adtagroup1/batch_processing/noise_complaints_20240417.csv noise_complaints.csv

Copying gs://adtagroup1/batch_processing/noise_complaints_20240417.csv...
/ [1 files][ 1010 KiB/ 1010 KiB]                                                
Operation completed over 1 objects/1010.4 KiB.                                   


In [18]:
! hdfs dfs -put noise_complaints.csv /user/adta5240grp/data

In [19]:
! hdfs dfs -ls /user/adta5240grp/data

Found 1 items
-rw-r--r--   2 root hadoop    1034613 2024-04-17 19:24 /user/adta5240grp/data/noise_complaints.csv


In [20]:
spark

In [93]:
df = spark.read.csv('gs://adtagroup1/batch_processing/noise_complaints_20240417.csv', header=True, inferSchema=True)


In [94]:
df.show()


+---+--------+-------------------+-------------------+---------+--------------------+-------+
|_c0| case_no|          open_date|         close_date|case_year|            sub_type|address|
+---+--------+-------------------+-------------------+---------+--------------------+-------+
|  0|20241528|2024-04-16 14:29:24|               null|     2024|Residential   El ...|  20871|
|  1|20241526|2024-04-16 08:00:25|               null|     2024|Industrial/commer...|  20910|
|  2|20241523|2024-04-16 06:25:00|2024-04-16 09:27:27|     2024|Industrial/commer...|  20910|
|  3|20241522|2024-04-15 17:13:40|               null|     2024|Residential   El ...|  20814|
|  4|20241517|2024-04-15 10:17:44|               null|     2024|Industrial/commer...|  20902|
|  5|20241516|2024-04-15 09:15:11|               null|     2024|Industrial/commer...|  20910|
|  6|20241515|2024-04-15 09:07:38|               null|     2024|Industrial/commer...|  20886|
|  7|20241514|2024-04-15 09:03:12|               null|     2

In [95]:
from pyspark.sql.functions import col, to_date

df = df.withColumn("open_date", to_date(col("open_date"), "MM/dd/yyyy")) \
       .withColumn("close_date", to_date(col("close_date"), "MM/dd/yyyy"))
        

In [96]:
df = df.withColumnRenamed('_c0', 'serial_no') \
       .withColumnRenamed('address', 'address_zipcode')


In [97]:
df.dtypes

[('serial_no', 'int'),
 ('case_no', 'int'),
 ('open_date', 'date'),
 ('close_date', 'date'),
 ('case_year', 'int'),
 ('sub_type', 'string'),
 ('address_zipcode', 'string')]

In [98]:
df.filter(df["close_date"].isNull()).show()

+---------+--------+----------+----------+---------+--------------------+---------------+
|serial_no| case_no| open_date|close_date|case_year|            sub_type|address_zipcode|
+---------+--------+----------+----------+---------+--------------------+---------------+
|        0|20241528|2024-04-16|      null|     2024|Residential   El ...|          20871|
|        1|20241526|2024-04-16|      null|     2024|Industrial/commer...|          20910|
|        3|20241522|2024-04-15|      null|     2024|Residential   El ...|          20814|
|        4|20241517|2024-04-15|      null|     2024|Industrial/commer...|          20902|
|        5|20241516|2024-04-15|      null|     2024|Industrial/commer...|          20910|
|        6|20241515|2024-04-15|      null|     2024|Industrial/commer...|          20886|
|        7|20241514|2024-04-15|      null|     2024|Industrial/commer...|          20814|
|        8|20241511|2024-04-12|      null|     2024|Residential   Con...|          20837|
|        9

df.filter(df["address_zipcode"].isNull()).show()

# Replace null values in 'Address Zip Code' with 'Unknown'

In [99]:
df = df.fillna({'address_zipcode': 'unknown'}) \
        .fillna({'address_zipcode': 'unknown'})


In [100]:
df.show()

+---------+--------+----------+----------+---------+--------------------+---------------+
|serial_no| case_no| open_date|close_date|case_year|            sub_type|address_zipcode|
+---------+--------+----------+----------+---------+--------------------+---------------+
|        0|20241528|2024-04-16|      null|     2024|Residential   El ...|          20871|
|        1|20241526|2024-04-16|      null|     2024|Industrial/commer...|          20910|
|        2|20241523|2024-04-16|2024-04-16|     2024|Industrial/commer...|          20910|
|        3|20241522|2024-04-15|      null|     2024|Residential   El ...|          20814|
|        4|20241517|2024-04-15|      null|     2024|Industrial/commer...|          20902|
|        5|20241516|2024-04-15|      null|     2024|Industrial/commer...|          20910|
|        6|20241515|2024-04-15|      null|     2024|Industrial/commer...|          20886|
|        7|20241514|2024-04-15|      null|     2024|Industrial/commer...|          20814|
|        8

In [101]:
df_null_close_date = df.filter(col("close_date").isNull())
df_null_close_date.show()

+---------+--------+----------+----------+---------+--------------------+---------------+
|serial_no| case_no| open_date|close_date|case_year|            sub_type|address_zipcode|
+---------+--------+----------+----------+---------+--------------------+---------------+
|        0|20241528|2024-04-16|      null|     2024|Residential   El ...|          20871|
|        1|20241526|2024-04-16|      null|     2024|Industrial/commer...|          20910|
|        3|20241522|2024-04-15|      null|     2024|Residential   El ...|          20814|
|        4|20241517|2024-04-15|      null|     2024|Industrial/commer...|          20902|
|        5|20241516|2024-04-15|      null|     2024|Industrial/commer...|          20910|
|        6|20241515|2024-04-15|      null|     2024|Industrial/commer...|          20886|
|        7|20241514|2024-04-15|      null|     2024|Industrial/commer...|          20814|
|        8|20241511|2024-04-12|      null|     2024|Residential   Con...|          20837|
|        9

In [102]:
df_null_close_date.count()

84

In [103]:
df = df.filter(col("close_date").isNotNull())

In [104]:
df.show()

+---------+--------+----------+----------+---------+--------------------+---------------+
|serial_no| case_no| open_date|close_date|case_year|            sub_type|address_zipcode|
+---------+--------+----------+----------+---------+--------------------+---------------+
|        2|20241523|2024-04-16|2024-04-16|     2024|Industrial/commer...|          20910|
|       28|20241446|2024-03-28|2024-04-03|     2024|Residential   Con...|          20895|
|       32|20241432|2024-03-26|2024-04-10|     2024|Industrial/commer...|          20877|
|       36|20241425|2024-03-25|2024-04-03|     2024|Residential   El ...|          20814|
|       37|20241422|2024-03-25|2024-03-27|     2024|Residential   El ...|          20833|
|       41|20241398|2024-03-20|2024-04-16|     2024|Industrial/commer...|          20852|
|       52|20241336|2024-03-11|2024-04-12|     2024|Residential   Con...|          20895|
|       54|20241332|2024-03-11|2024-04-16|     2024|Residential   Mec...|          20877|
|       57

In [105]:
df.count()

9368

In [106]:
import os

# Point to the JSON key file of your service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gs://adtagroup1/key/keys.json'

# Define the GCS bucket path
output_path = 'gs://adtagroup1/spark_tranformed/'

df.write.csv(output_path, mode='overwrite', header=True)